In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf 
from scipy import stats
import warnings

In [2]:
sns.set(rc={'savefig.dpi':200})
sns.set_context('notebook', rc={'lines.linewidth':2})
sns.set_style('darkgrid')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.rcParams['figure.dpi'] = 100
plt.rcParams["figure.autolayout"] = True

/var/folders/q1/m9fsct652l59y6ctvtwrwg7c0000gn/T/ipykernel_61376/2172238296.py:5: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [84]:
df = pd.read_csv('./project_data_unified/officers_only.csv')

In [85]:
# IGNORE THIS CELL 
x = df.groupby('disciplined')['unit_description'].value_counts().index

# Extract just the unit_descriptions for each of discplined = 1 and discplined = 0
y_disc = []
n_disc = []
for i in range(len(x)):
    if x[i][0] == 0:
        n_disc.append(x[i][1])
    else:
        y_disc.append(x[i][1])

# Find which unit_descriptions are NOT SHARED between discplined =1 and discplined = 0
uncommon = set(y_disc) ^ set(n_disc)

In [133]:
# Ignore regular expression warning messages
warnings.filterwarnings('ignore')

# DUMMIES::

# unit_description dummy df
unit_df = (pd.get_dummies(df['unit_description'])).astype(float)
unit_df = unit_df.drop(columns='Unknown')
# complaint_category dummy df
complaint_df = (pd.get_dummies(df['complaint_category'])).astype(float)
complaint_df = complaint_df.drop(columns='Other')
# officer gender dummy 
gender_df = pd.get_dummies(df['officer_gender'])
# drop female (only need 1 dummy to encode binary)
gender_df = gender_df.drop(columns='FEMALE')
# officer Race dummy 
race_df = pd.get_dummies(df['officer_race'])
race_df.columns = race_df.columns.str.replace('/','_')

# Clean up columns
complaint_df.columns = complaint_df.columns.str.replace(' ', '_')
complaint_df.columns = complaint_df.columns.str.replace('/', '_')
complaint_df.columns = complaint_df.columns.str.replace(')', '')
complaint_df.columns = complaint_df.columns.str.replace('(', '_')
complaint_df.columns = complaint_df.columns.str.replace('-', '_')

unit_df.columns = unit_df.columns.str.replace(' ', '_')
unit_df.columns = unit_df.columns.str.replace('-', '_')
unit_df.columns = unit_df.columns.str.replace('(', '')
unit_df.columns = unit_df.columns.str.replace(')', '')
unit_df.columns = unit_df.columns.str.replace('.', '')
unit_df.columns = unit_df.columns.str.replace('&', '_')

# Combine complaint_category + unit_description data frames 
complaint_unit_df = pd.concat([complaint_df, unit_df], axis=1)
# Combine gender and race 
gender_race_df = pd.concat([race_df, gender_df], axis=1)

# complaint_category only 

In [124]:
Xs = '+'.join(complaint_df.columns.difference(['disciplined']))
complaint_df['disciplined'] = df['disciplined']
logit_mod = smf.logit('disciplined~{}'.format(Xs),data=complaint_df).fit()
print('!!COMPLAINT CATEGORY ONLY!!')
print(logit_mod.summary())

Optimization terminated successfully.
         Current function value: 0.164712
         Iterations 10
!!COMPLAINT CATEGORY ONLY!!
                           Logit Regression Results                           
Dep. Variable:            disciplined   No. Observations:               194142
Model:                          Logit   Df Residuals:                   194128
Method:                           MLE   Df Model:                           13
Date:                Mon, 27 Feb 2023   Pseudo R-squ.:                 0.09287
Time:                        15:17:50   Log-Likelihood:                -31977.
converged:                       True   LL-Null:                       -35251.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         -4.58

# unit_description only

In [125]:
Xs = '+'.join(unit_df.columns)
unit_df['disciplined'] = df['disciplined']
logit_mod = smf.logit('disciplined~{}'.format(Xs),data=unit_df).fit()
print('!!COMPLAINT CATEGORY ONLY!!')
print(logit_mod.summary())

Optimization terminated successfully.
         Current function value: 0.178805
         Iterations 9
!!COMPLAINT CATEGORY ONLY!!
                           Logit Regression Results                           
Dep. Variable:            disciplined   No. Observations:               194142
Model:                          Logit   Df Residuals:                   194074
Method:                           MLE   Df Model:                           67
Date:                Mon, 27 Feb 2023   Pseudo R-squ.:                 0.01525
Time:                        15:19:45   Log-Likelihood:                -34714.
converged:                       True   LL-Null:                       -35251.
Covariance Type:            nonrobust   LLR p-value:                1.206e-181
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept         

# complaint_category + unit_description 

In [6]:
# COMPLAINT + UNIT MODEL
Xs = '+'.join(complaint_unit_df.columns)
complaint_unit_df['disciplined'] = df['disciplined']
logit_mod = smf.logit('disciplined~{}'.format(Xs),data=complaint_unit_df).fit()
print('!!COMPLAINT + UNIT MODEL!!')
print(logit_mod.summary())

Optimization terminated successfully.
         Current function value: 0.163120
         Iterations 10
COMPLAINT + UNIT MODEL
                           Logit Regression Results                           
Dep. Variable:            disciplined   No. Observations:               194142
Model:                          Logit   Df Residuals:                   194061
Method:                           MLE   Df Model:                           80
Date:                Mon, 27 Feb 2023   Pseudo R-squ.:                  0.1016
Time:                        13:54:16   Log-Likelihood:                -31668.
converged:                       True   LL-Null:                       -35251.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept             

# officer_gender + officer_race

In [126]:
Xs = '+'.join(gender_race_df.columns.difference(['discplined']))
gender_race_df['disciplined'] = df['disciplined']
logit_mod = smf.logit('disciplined~{}'.format(Xs),data=gender_race_df).fit()
print('!!COMPLAINT + UNIT MODEL!!')
print(logit_mod.summary())

Optimization terminated successfully.
         Current function value: 0.177239
         Iterations 7
!!COMPLAINT + UNIT MODEL!!
                           Logit Regression Results                           
Dep. Variable:            disciplined   No. Observations:               194142
Model:                          Logit   Df Residuals:                   194136
Method:                           MLE   Df Model:                            5
Date:                Mon, 27 Feb 2023   Pseudo R-squ.:                 0.02388
Time:                        15:25:00   Log-Likelihood:                -34409.
converged:                       True   LL-Null:                       -35251.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.6946      0.363     -4.673      0.000      

# complaint + unit + race + gender

In [136]:
# COMPLAINT + UNIT + RACE + GENDER MODEL
to_concat = [unit_df, complaint_df, gender_df, race_df]
gender_race_unit_complaint_df = pd.concat(to_concat, axis=1)
gender_race_unit_complaint_df['disciplined'] = df['disciplined']
Xs = '+'.join(gender_race_unit_complaint_df.columns.difference(['disciplined']))
logit_mod = smf.logit('disciplined~{}'.format(Xs), data = gender_race_unit_complaint_df)
logit_mod = logit_mod.fit(maxiter=300)
print('COMPLAINT + UNIT + RACE + GENDER MODEL')
print(logit_mod.summary())

Optimization terminated successfully.
         Current function value: 0.161312
         Iterations 10
COMPLAINT + UNIT + RACE + GENDER MODEL
                           Logit Regression Results                           
Dep. Variable:            disciplined   No. Observations:               194142
Model:                          Logit   Df Residuals:                   194056
Method:                           MLE   Df Model:                           85
Date:                Mon, 27 Feb 2023   Pseudo R-squ.:                  0.1116
Time:                        15:28:27   Log-Likelihood:                -31317.
converged:                       True   LL-Null:                       -35251.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Interc

# FULL MODEL (excluding rank)

In [92]:
gender_race_unit_complaint_df = pd.concat([gender_race_df, complaint_unit_df], axis=1)
duty_df = pd.get_dummies(df['on_off_duty'])
# Drop OFF dummy (only need 1 to encode binary)
duty_df = duty_df.drop(columns='OFF')
# Create full data frame 
full_df = pd.concat([gender_race_unit_complaint_df, duty_df], axis=1)
# Add some noise to salary
full_df['salary'] = (df['salary'] + np.random.normal(0,5,df.shape[0]))
# Add some noise to age
full_df['officer_age'] = df['officer_age'] + np.random.normal(0,1.5,df.shape[0])

In [99]:
Xs = '+'.join(full_df.columns.difference(['disciplined']))
full_df['disciplined'] = df['disciplined']
#gender_race_unit_complaint_df.iloc[:,-1] = gender_race_unit_complaint_df.iloc[:,-1] + np.random.normal(0, 0.002, 194142)
logit_mod = smf.logit('disciplined~{}'.format(Xs), data = full_df)
logit_mod = logit_mod.fit(maxiter=500)
print('FULL MODEL')
print(logit_mod.summary())

Optimization terminated successfully.
         Current function value: 0.160448
         Iterations 10
FULL MODEL
                           Logit Regression Results                           
Dep. Variable:            disciplined   No. Observations:               194142
Model:                          Logit   Df Residuals:                   194053
Method:                           MLE   Df Model:                           88
Date:                Mon, 27 Feb 2023   Pseudo R-squ.:                  0.1164
Time:                        14:56:56   Log-Likelihood:                -31150.
converged:                       True   LL-Null:                       -35251.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                         

# backward-selection

In [122]:
X = list(full_df.columns.difference(['disciplined']))
Xs = '+'.join(X)
logit_model = smf.logit('disciplined~{}'.format(Xs), data = full_df).fit()

while True:
    max_pvalue = max(logit_model.pvalues)
    if max_pvalue > 0.05:
        # remove the predictor with the largest p-value
        predictor_to_remove = logit_model.pvalues.idxmax()
        X.remove(predictor_to_remove)
        Xs = '+'.join(X)
        logit_model = smf.logit('disciplined~{}'.format(Xs), data=full_df).fit(disp=False)
    else:
        break
        
final_back_selected_mod = logit_model
print(final_back_selected_mod.summary())

Optimization terminated successfully.
         Current function value: 0.160448
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:            disciplined   No. Observations:               194142
Model:                          Logit   Df Residuals:                   194095
Method:                           MLE   Df Model:                           46
Date:                Mon, 27 Feb 2023   Pseudo R-squ.:                  0.1153
Time:                        15:14:13   Log-Likelihood:                -31187.
converged:                       True   LL-Null:                       -35251.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                -3.7753     

# rank and current_rank

In [95]:
df['rank'] = df['rank'].replace('DEPUTY CHIEF', 'CHIEF')
df['rank'] = df['rank'].replace('CHIEF', 'CHIEF_OR_DEPUTY')

df['current_rank'] = df['current_rank'].astype(str)
df['current_rank'] = df['current_rank'].replace('UNKNOWN', 'Other')

In [96]:
# FULL MODEL
rank_df = pd.get_dummies(df['rank'])
rank_df.columns = rank_df.columns.str.replace(' ','_')
current_rank_df = pd.get_dummies(df['current_rank'])
current_rank_df.columns = current_rank_df.columns.str.replace(' ','_')

In [177]:
Xs = '+'.join(current_rank_df.columns.difference(['disciplined']))
#current_rank_df['disciplined'] = df['disciplined']
logit_mod = smf.logit('disciplined~{}'.format(Xs),data=current_rank_df).fit(maxiter=100)
print('COMPLAINT + UNIT MODEL')
print(logit_mod.summary())

         Current function value: 0.181228
         Iterations: 100
COMPLAINT + UNIT MODEL
                           Logit Regression Results                           
Dep. Variable:            disciplined   No. Observations:               194142
Model:                          Logit   Df Residuals:                   194136
Method:                           MLE   Df Model:                            5
Date:                Mon, 27 Feb 2023   Pseudo R-squ.:                0.001908
Time:                        18:45:15   Log-Likelihood:                -35184.
converged:                      False   LL-Null:                       -35251.
Covariance Type:            nonrobust   LLR p-value:                 2.624e-27
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.9346   6.05e+05  -4.85e-06      1.000   -1.19e+06    1.19e+06
CHIEF              -1.2875

In [171]:
df.groupby('disciplined')['rank'].value_counts()

disciplined  rank                   
0            POLICE OTHER               157632
             SERGEANT                    16929
             PO AS DETECTIVE              9720
             COMMANDER                     757
             LIEUTENANT                    205
             Other                         110
             EXPLOSIVES TECHNICIAN I       103
             CHIEF_OR_DEPUTY                73
1            POLICE OTHER                 7471
             SERGEANT                      737
             PO AS DETECTIVE               360
             COMMANDER                      20
             LIEUTENANT                      9
             CHIEF_OR_DEPUTY                 6
             EXPLOSIVES TECHNICIAN I         6
             Other                           4
Name: rank, dtype: int64

In [172]:
df.groupby('disciplined')['current_rank'].value_counts()

disciplined  current_rank   
0            POLICE OFFICER     161662
             PO AS DETECTIVE     21269
             COMMANDER            1217
             CHIEF                 750
             Other                 631
1            POLICE OFFICER       7828
             PO AS DETECTIVE       720
             Other                  30
             COMMANDER              24
             CHIEF                  11
Name: current_rank, dtype: int64

In [ ]:

to_concat = [unit_df, complaint_df]
X = pd.concat(to_concat, axis=1)
y = df['disciplined']

In [ ]:
logit_obj = sm.Logit(y, sm.add_constant(complaint_df))
model = logit_obj.fit(maxiter=500)
print(model.summary())

In [ ]:
logit_mod = smf.logit('disciplined~complaint_category+officer_race',data=df).fit()
print(logit_mod.summary())